In [1]:
import sys,os
#os.chdir("ecg")
#import keras as keras
import tensorflow as tf
import json
import load
import network
from tensorflow.keras.optimizers import Adam
import numpy as np
import collections
%load_ext autoreload
%autoreload 2
os.chdir("/workspace/telemed5000/ecg/")

In [2]:
tf.keras.__version__

'2.2.4-tf'

In [16]:
data_json, model_path = "examples/cinc17/dev.json","../saved/cinc17/1575022999-481/0.384-0.890-019-0.301-0.898.hdf5"
#model = keras.models.load_model(model_path)
#

In [25]:
params = json.load(open("examples/cinc17/config.json", 'r'))
print("Loading training set...")
train = load.load_dataset(params['train'])
print("Loading dev set...")
dev = load.load_dataset(params['dev'])
print("Building preprocessor...")
preproc = load.Preproc(*train)
print("Training size: " + str(len(train[0])) + " examples.")
print("Dev size: " + str(len(dev[0])) + " examples.")
params.update({
    "input_shape": [None, 1],
    "num_categories": len(preproc.classes)
})
with tf.device('/gpu:3'):
    model = network.build_network(**params)

  5%|▌         | 396/7676 [00:00<00:01, 3950.88it/s]

Loading training set...


 49%|████▊     | 415/852 [00:00<00:00, 4142.94it/s]

Loading dev set...


100%|██████████| 852/852 [00:00<00:00, 4118.19it/s]


Building preprocessor...
Training size: 7676 examples.
Dev size: 852 examples.


In [34]:
train_x, train_y = preproc.process(*train)
dev_x, dev_y = preproc.process(*dev)

In [37]:
train_y.shape

(7676, 71, 4)

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, None, 32)     544         inputs[0][0]                     
__________________________________________________________________________________________________
batch_normalization_33 (BatchNo (None, None, 32)     128         conv1d_33[0][0]                  
__________________________________________________________________________________________________
activation_34 (Activation)      (None, None, 32)     0           batch_normalization_33[0][0]     
____________________________________________________________________________________________

In [26]:
with tf.device('/gpu:3'):
    optimizer = Adam(
            lr=params["learning_rate"],
            clipnorm=params.get("clipnorm", 1))

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

In [27]:
with tf.device('/gpu:3'):
    model.fit(train_x, train_y)

7676/7676 [==============================] - 51s 7ms/sample - loss: 0.4982 - acc: 0.7937


In [22]:
model.get_weights()

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [24]:
def preprocess(dataset):

  def element_fn(element):
    return collections.OrderedDict([
        ('x', element[0]),
        ('y', element[1]),
    ])

  return dataset.repeat(NUM_EPOCHS).map(element_fn).shuffle(
      SHUFFLE_BUFFER).batch(BATCH_SIZE)

In [30]:
preprocessed_example_dataset = preprocess(tf.data.Dataset.from_generator(lambda: loader.clientsData,tf.float64))

In [31]:
sample_batch = tf.nest.map_structure(
    lambda x: x, next(iter(loader.clientsData)))

In [5]:
NUM_CLIENTS = 10
NUM_EPOCHS = 10
BATCH_SIZE = 20
SHUFFLE_BUFFER = 500

def preprocess(dataset):

  def element_fn(element):
    return collections.OrderedDict([
        ('x', train_x),
        ('y', train_y),
    ])

  return dataset.repeat(NUM_EPOCHS).map(element_fn).shuffle(
      SHUFFLE_BUFFER).batch(BATCH_SIZE)

In [32]:
from tensorflow.compat.v2 import convert_to_tensor
# Wrap a Keras model for use with TFF.
def model_fn():
    model = network.build_network(**params)
    optimizer = Adam(
        lr=params["learning_rate"],
        clipnorm=params.get("clipnorm", 1))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=optimizer,
        metrics=[tf.keras.metrics.Accuracy()])
    return tff.learning.from_compiled_keras_model(model, convert_to_tensor(sample_batch))

In [33]:
model_fn()

ValueError: Can't convert non-rectangular Python sequence to Tensor.

iterative_process = tff.learning.build_federated_averaging_process(model_fn)
state = iterative_process.initialize()
for round_num in range(2, 11):
  state, metrics = iterative_process.next(state, loader.clientsData)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

In [18]:
import tensorflow as tf
tf.compat.v1.enable_v2_behavior()
import tensorflow_federated as tff

# Load simulation data.
source, _ = tff.simulation.datasets.emnist.load_data()
def client_data(n):
  return source.create_tf_dataset_for_client(source.client_ids[n]).map(
      lambda e: {
          'x': tf.reshape(e['pixels'], [-1]),
          'y': e['label'],
  }).repeat(10).batch(20)

# Pick a subset of client devices to participate in training.

/usr/local/lib/python3.6/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [3]:
train_data = [client_data(n) for n in range(3)]

# Grab a single batch of data so that TFF knows what data looks like.
sample_batch = tf.nest.map_structure(
    lambda x: x.numpy(), iter(train_data[0]).next())

W1206 10:47:37.859699 140048581662528 ag_logging.py:146] Entity <function client_data.<locals>.<lambda> at 0x7f5f5ac04bf8> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Failed to parse source code of <function client_data.<locals>.<lambda> at 0x7f5f5ac04bf8>, which Python reported as:
      lambda e: {
          'x': tf.reshape(e['pixels'], [-1]),
          'y': e['label'],
  }).repeat(10).batch(20)

This may be caused by multiline strings or comments not indented at the same level as the code.
W1206 10:47:37.886264 140048581662528 ag_logging.py:146] Entity <function client_data.<locals>.<lambda> at 0x7f5f8d2e6048> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full 

      lambda e: {
          'x': tf.reshape(e['pixels'], [-1]),
          'y': e['label'],
  }).repeat(10).batch(20)

This may be caused by multiline strings or comments not indented at the same level as the code.
      lambda e: {
          'x': tf.reshape(e['pixels'], [-1]),
          'y': e['label'],
  }).repeat(10).batch(20)

This may be caused by multiline strings or comments not indented at the same level as the code.
      lambda e: {
          'x': tf.reshape(e['pixels'], [-1]),
          'y': e['label'],
  }).repeat(10).batch(20)

This may be caused by multiline strings or comments not indented at the same level as the code.


In [3]:
from keras import backend
len(backend.tensorflow_backend._get_available_gpus())

0

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [1]:
class FederatedDataLoader:
    
    def __init__(self, data_json, clients=10):
        self.allData = load.load_dataset(data_json)
        self.clients = clients
        self.groupClientData()
    
    def clientData(self, index):
        return self.clientsData[index]
    
    def groupClientData(self):
        self.clientsData = []
        xSplit = self.splitArray(self.allData[0])
        ySplit = self.splitArray(self.allData[1])
        
        for index in range(self.clients):
            self.clientsData.append([xSplit[index], ySplit[index]])

    def splitArray(self, array):
        splitLen = len(array)/self.clients
        return np.array_split(array, splitLen)
loader = FederatedDataLoader("examples/cinc17/dev.json")

NameError: name 'load' is not defined